In [1]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()  # 환경 변수 로드

False

In [2]:
# 사용자 인증을 통해 API 토큰을 얻음. 이 토큰은 이후 API 호출에서 사용됨.
def authenticate():
    url = 'https://bmt.eurekachat.ai:8443/api/v2/auth/login'
    payload = {
        'username': os.getenv('USERNAME'),
        'password': os.getenv('PASSWORD'),
        'tenant_id': os.getenv('TENANT_ID')
    }
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    response = requests.post(url, data=payload, headers=headers)
    
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        print(f"Failed to authenticate: {response.status_code}")
        return None

token = authenticate()
print(token)

Failed to authenticate: 422
None


In [3]:
# 등록된 문서 확인
def get_documents(auth_token):
    url = 'https://bmt.eurekachat.ai:8443/api/v2/docs/infos/manage/'
    headers = {
        'accept': 'application/json',
        'Authorization': f'{auth_token}'
    }
    # print(auth_token)
    response = requests.get(url, headers=headers)
    print(response.json())
    if response.status_code == 200:
        documents = response.json()['list']
        for doc in documents:
            print(f"Document: {doc['name']}, Status: {doc['status']}")
    else:
        print(f"Failed to retrieve documents: {response.status_code}")

# Example usage:
token = authenticate()
if token:
    get_documents(token)


Failed to authenticate: 422


In [4]:
def query_documents(auth_token, query, docs_ids):
    url = 'https://bmt.eurekachat.ai:8443/api/v2/indices/chat?use_memory=false&use_agent=false&use_merge=false&stream=true'
    headers = {
        'accept': 'application/json',
        'Authorization': f'{auth_token}',
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    payload = {
        'query': query,
        'docs_id': ','.join(docs_ids),
        'tenant_opts_id': '4ac3a1b30e0a051171a5bf7fd051ebcf2a7da6b0b98bbdac0ef5e347223cd777'
    }
    
    response = requests.post(url, data=payload, headers=headers)
    
    if response.status_code == 200:
        print(response.json())
    else:
        print(f"Failed to query documents: {response.status_code}")

# Example usage:
query = "이 상품의 주요 특징에 대해 간단히 알려주세요"
docs_ids = ['313df4e36d112f233e533f14b0b1a4da65ac3acc6528e41f8511887f35d90d47',
            '49976be3b043e2eb696c00e44d246b1462321178399659a691623c69bb6d422d']
token = authenticate()
if token:
    query_documents(token, query, docs_ids)


Failed to authenticate: 422


In [5]:
# 여러 건의 문서 등록
def register_documents(auth_token, documents):
    url = 'https://bmt.eurekachat.ai:8443/api/v2/docs/infos/'
    headers = {
        'accept': 'application/json',
        'Authorization': f'{auth_token}'
    }
    
    for document in documents:
        files = {
            'name': (None, document['name']),
            'desc': (None, document['desc']),
            'category_id': (None, str(document['category_id'])),
            'status': (None, document['status']),
            'keyword': (None, document.get('keyword', '')),
            'product_name': (None, document.get('product_name', '')),
            'opts_id': (None, document['opts_id'])
        }
        
        response = requests.post(url, headers=headers, files=files)
        
        if response.status_code == 200:
            print(f"Document '{document['name']}' registered successfully.")
        else:
            print(f"Failed to register document '{document['name']}': {response.status_code}")

# Example usage:
documents = [
    {
        'name': 'Document 1',
        'desc': 'Description of document 1',
        'category_id': 7,
        'status': 'DONE',
        'opts_id': '4ac3a1b30e0a051171a5bf7fd051ebcf2a7da6b0b98bbdac0ef5e347223cd777'
    },
    {
        'name': 'Document 2',
        'desc': 'Description of document 2',
        'category_id': 9,
        'status': 'DONE',
        'opts_id': 'e3e2ec97dc1ee5053f00492ea0181e1c2637d39874c1d7c2986ef408faed9389'
    }
]

token = authenticate()
if token:
    register_documents(token, documents)

Failed to authenticate: 422
